In [ ]:
import numpy as np
from TMDP import TMDP
from algorithms import *
from model_functions import *

import matplotlib.pyplot as plt
import gymnasium as gym
from FrozenLake import *

#np.set_printoptions(precision=4)
import math

import torch
import mlflow
from CurriculumPMPO import CurriculumPMPO
import optuna
from policy_utils import *
from constants import *
from experiment_result_utils import *

nrows = 30
nS = nrows**2
nA = 4
map_seed = constants.SEEDS[6]


gamma = .99999999

is_slippery = False
debug = False
num_bins = 0
if num_bins == 0:
    reward_shape = dense_reward = False
else:
    reward_shape = dense_reward = True

checkpoint_step=500
test_episodes = 1000
episodes = 3900000
num_runs = 60
check_convergence = True

shape_range=(-1,0)

env = FrozenLakeEnv(is_slippery=False, seed=None, 
                    desc=generate_random_map(nrows, seed=map_seed), 
                    reward_shape=reward_shape,
                    num_bins=num_bins,
                    goal_reward=1,
                    shape_range=shape_range,
                    dense_reward=dense_reward,
                    )

xi = np.ones(env.nS) * 1/env.nS
tau = 1.
tmdp = TMDP(env, xi, tau=tau, gamma=gamma, seed=None)

In [ ]:

def get_or_create_experiment(experiment_name):
    if experiment := mlflow.get_experiment_by_name(experiment_name):
        return experiment.experiment_id
    else:
        return mlflow.create_experiment(experiment_name)


In [ ]:
mlflow.set_tracking_uri(MLFLOW_URI)
experiment_name = "Frozen_Lake_{}_{}_{} OPTUNA".format(nrows, nrows, num_bins)
experiment_id = get_or_create_experiment(experiment_name)
mlflow.set_experiment(experiment_name)

In [ ]:
final_temp=1e-3
param_decay=True
lam = 1
epochs = 4
eps_ppo = 0.2


In [ ]:
from experiment_result_utils import *

In [ ]:

def objective(trial):
    with mlflow.start_run(nested=True):
        global episodes
        # Parameter space optimization
        model_lr = trial.suggest_float("model_lr", 1e-4, 0.5, log=True)
        pol_lr = trial.suggest_float("pol_lr", 1e-4, 0.5, log=True)
        entropy_coef = trial.suggest_float("entropy_coef", 1e-4, 0.5, log=True)
        batch_size = trial.suggest_int("batch_size", 1, 32, log=False)
        #epochs = trial.suggest_int("epochs", 1, 5, log=False)
        #lam = trial.suggest_float("lam", 0., 1., log=False)
        temp = trial.suggest_float("temp", 1, 20, log=False)
        #param_decay = trial.suggest_categorical("param_decay", [True, False])
        eps_shift = trial.suggest_float("eps_shift", 1e-5, 6e-3, log=True)
        tau = trial.suggest_float("tau", 0.2, 0.5, log=False)
        
        
        set_policy_seed(None)
        
        # Curriculum class initialization
        tmdp.reset()
        tmdp.update_tau(tau) # Reset the teleport rate 
        max_eps_model = eps_shift*0.5
        curr_PMPO = CurriculumPMPO(tmdp, checkpoint_step=checkpoint_step)
        curr_PMPO.train(model_lr, pol_lr, batch_size=batch_size, 
                lam=lam, temp=temp, final_temp=final_temp,
                episodes=episodes, eps_ppo=eps_ppo,
                eps_shift=eps_shift,
                param_decay=param_decay, epochs=epochs, max_length=1000,
                entropy_coef=entropy_coef, debug=debug,
                max_eps_model=max_eps_model,
                check_convergence = check_convergence)
        
        
        avg_reward = np.mean(curr_PMPO.reward_records[-1:])/batch_size
        
        test_pol = test_policies_len(tmdp, [curr_PMPO.theta], test_episodes, 1e-20)
        cum_return = test_pol[0][0]
        ep_len = test_pol[1][0]
        mlflow.log_params({
            "model_lr": model_lr,
            "pol_lr": pol_lr,
            "temp": temp,
            "entropy_coef": entropy_coef,
            "param_decay": param_decay,
            "eps_shift": eps_shift,
            "tau": tau,
        })

        mlflow.log_metric("Avg Reward", avg_reward)
        mlflow.log_metric("Cumulative Return", cum_return)
        mlflow.log_metric("Episode Length", ep_len)
        mlflow.set_tags(tags={
            "eps_ppo": eps_ppo,
            "lam": lam,
            "batch_size": batch_size,
            "epochs": epochs,
            "final_temp": final_temp,
            "max_eps_model": max_eps_model,
        })

    return cum_return


In [ ]:
run_name = "CPMPO"
with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True):
    if num_bins == 0:
        study = optuna.create_study(direction="maximize")
    else:
        study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=num_runs)
    
    # Train and save the best model
    curr_PMPO = CurriculumPMPO(tmdp, checkpoint_step=checkpoint_step)
        
    # Get the best parameters
    model_lr = study.best_params["model_lr"]
    pol_lr = study.best_params["pol_lr"]
    batch_size = study.best_params["batch_size"]
    #lam = study.best_params["lam"]
    temp = study.best_params["temp"]
    #epochs = study.best_params["epochs"]
    entropy_coef = study.best_params["entropy_coef"]
    #param_decay = study.best_params["param_decay"]
    eps_shift = study.best_params["eps_shift"]
    tau = study.best_params["tau"]
    

    tmdp.update_tau(tau) # Reset the teleport rate 
    # Train the best model
    max_eps_model = eps_shift*0.5

    curr_PMPO.train(model_lr, pol_lr, batch_size=batch_size, 
                lam=lam, temp=temp, final_temp=final_temp,
                episodes=episodes, eps_ppo=eps_ppo,
                eps_shift=eps_shift,
                param_decay=param_decay, epochs=epochs, max_length=1000,
                entropy_coef=entropy_coef, debug=debug,
                max_eps_model=max_eps_model)
    
    mlflow.log_params({
            
        })

    mlflow.set_tags(tags={
            "Algorithm": "CurriculumPPO",
            "nS": nS,
            "nA": nA,
            "gamma": gamma,
            "seed": seed,
            "tau": tau,
            "model_lr": model_lr,
            "pol_lr": pol_lr,
            "batch_size": batch_size,
            "lam": lam,
            "temp": temp,
            "final_temp": final_temp,
            "episodes": episodes,
            "eps_ppo": eps_ppo,
            "eps_model": eps_shift,
            "entropy_coef": entropy_coef,
            "epochs": epochs,
            "max_eps_model": max_eps_model,
            "num_bins": num_bins,

        })

In [ ]:
print(curr_PMPO.theta)


In [ ]:
import matplotlib.pyplot as plt
reward_records = curr_PMPO.reward_records
# Generate recent 50 interval average
average_reward = []
scaled_reward = []
for idx in range(len(reward_records)):
    avg_list = np.empty(shape=(1,), dtype=int)
    scaled_reward.append(reward_records[idx])
    if idx < 50:
        avg_list = reward_records[:idx+1]
    else:
        avg_list = reward_records[idx-49:idx+1]
    average_reward.append(np.average(avg_list))
# Plot
#plt.plot(scaled_reward)
plt.plot(average_reward[:-1])

In [ ]:
pi = get_softmax_policy(curr_PMPO.theta, 1e-40)
tmdp.update_tau(0.)
tmdp.reset()
done = False
step = 0
env.render_mode = "human"
while True:
    s = tmdp.env.s
    a = greedy(tmdp.env.s, pi, tmdp.env.allowed_actions[int(tmdp.env.s)])
    s_prime, reward, flags, prob = tmdp.step(a)
    print("state {} action {} next state {} reward {}".format(s, a, s_prime, reward))
    if flags["done"]:
        tmdp.reset()
    step +=1
    if step > max(100,nrows*2):
        break
